# prepDataTrainN

Simple algorithm to open shapefile, raster and cut files from one another using gdal

<b> Imports

In [1]:
import io
import os
import requests
import numpy as np
import pandas as pd
from osgeo import osr
from osgeo import gdal
from os import listdir
import geopandas as gpd
from os.path import isfile, join

<b> Set Paths

In [2]:
tiff_path = "/home/sansigolo/Documents/git/CAP-240-394/src/cut-areas/LC08_L1TP_221067_20170809_20170824_01_T1/"

shape_path = "//home/sansigolo/Documents/git/CAP-240-394/src/cut-areas/AQM_L8_221_067_2017/"

<b> Read the Files

In [4]:
filenames = [y for y in listdir(shape_path) for ending in ['dbf', 'shp', 'prj', 'shx'] if y.endswith(ending)] 
   
print("\nFilenames:",filenames)


Filenames: ['AQM_L8_221_067_2017.prj', 'AQM_L8_221_067_2017.shp', 'AQM_L8_221_067_2017.shx', 'AQM_L8_221_067_2017.dbf']


<b> Open Shapefile

In [6]:
dbf, shp, prj,  shx = [filename for filename in filenames]

df  = gpd.read_file(shape_path+shp)

print("\nShape of the dataframe: {}".format(df.shape)+"\n")


Shape of the dataframe: (5785, 36)



<b> Print Dataframe tail

In [7]:
print("\nDataframe tail:\n", df.tail())


Dataframe tail:
            id  cod_sat                cena_id                      nome_arq  \
5780  7102656        8  LC82210672017269LGN00  LC82210672017269LGN00.tar.gz   
5781  7097583        8  LC82210672017189LGN00  LC82210672017189LGN00.tar.gz   
5782  7098332        8  LC82210672017189LGN00  LC82210672017189LGN00.tar.gz   
5783  7097780        8  LC82210672017189LGN00  LC82210672017189LGN00.tar.gz   
5784  7097382        8  LC82210672017189LGN00  LC82210672017189LGN00.tar.gz   

                     data_pas  orb_pto    area_ha  perim versao  \
5780  2017/09/26 00:00:00.000  221_067   0.632092    420    3.3   
5781  2017/07/08 00:00:00.000  221_067   1.985133    840    3.3   
5782  2017/07/08 00:00:00.000  221_067  24.708948   7440    3.3   
5783  2017/07/08 00:00:00.000  221_067   0.992043    480    3.3   
5784  2017/07/08 00:00:00.000  221_067   0.541506    360    3.3   

                         n_arq_ant  \
5780  LC82210672017253LGN00.tar.gz   
5781  LC82210672017173LGN00.

<b> Filter the Dataframe

In [8]:
df = df[df['verifica'] == 3]
df = df[df['data_pas'] == '2017/08/09 00:00:00.000']

<b> Set Buffer

In [9]:
df['geometry'] = df['geometry'].buffer(0.002)
#df['geometry'] = df['geometry'].envelope

<b> For loop to crop all bands tifs from a from given Geometry

In [10]:
for i in range(9): #len(df)
    gp = df[df['geometry'] == df.iloc[i]['geometry']]
    gp.to_file(shape_path+"new_shape_221_067_20170809_"+str(i))
    shapefile = shape_path+"new_shape_221_067_20170809_"+str(i)+"/new_shape_221_067_20170809_"+str(i)+".shp"
    for j in range(1,12):
        new_tiff_file = "output/falsos_"+str(i)+"_B"+str(j)+"_.TIF"
        os.system("gdalwarp -overwrite -q -cutline " +shapefile+ " -crop_to_cutline -tr 30.0 30.0 " +tiff_path+'LC08_L1TP_221067_20170809_20170824_01_T1_B'+str(j)+'.TIF'+ " " +new_tiff_file)